<a href="https://colab.research.google.com/github/LM1997610/Networking-for-Big-Data/blob/main/NBD_HW2_scraps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class LWLDispatcher:
  def __init__(self, N):
    self.N = N
    self.controller = DispatcherControl(N)
    self.servers = [FCFS(id) for id in range(0,N)]
  def dispatch(self, task):
    workload = [sum(list(self.controller.details[s].values()))
                for s in range(0,self.N)]
    pointed = np.argmin(workload)
    self.servers[pointed].append(task)

class FCFS:
  def __init__(self, id):
    self.server_id = id 
    self.memory =  1 
    self.queue =  []  
    self.queue_status = []  
    self.workload = 0 
    #self.list_done_task = []   # for messages with dispatcher 

  def add_to_queue(self, dizionario : dict, µ = 0.1 ):
    new_job_id = str(int(dizionario["Job_ID"]))+ "_" + str(int(dizionario["Task_ID"]))
    arrival_time_task = int(dizionario["Arrival_Time"])
    running_time_task = (dizionario["CPU"]/ µ)*1000     # from seconds to milliseconds
    a_dict = {"id": new_job_id, "arrival_time": arrival_time_task, "service_time" : running_time_task}
    if self.memory > dizionario["Memory"]: 
      self.memory -= dizionario["Memory"]
      self.queue.append(a_dict)   ### and do some work     
    else: print("...not enough space in memory")  ## and think what to do
  pass

class JobControl:
  pass

class DispatcherControl:

  def __init__(self, N):
    self.servers_number = N
    self.status = {s:0 for s in range(0,N)}
    self.queue_length = {s:0 for s in range(0,N)}
    self.memory_status = {s:1 for s in range(0,N)}
    self.details = {s:({},0) for s in range(0,N)}

  def is_idle(self, server): 
    return self.status[server] == 0

  def memory_constraint(self, server, task_memory):
    return self.memory_status[server] - task_memory > 0

In [ ]:
disp = LWLDispatcher(64)